# **Convulotional Neural Networks**

The following code are inspired from
"Image Preparation for Convolutional Neural Networks with TensorFlow's Keras API" video by deeplizard

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter (action='ignore', category=FutureWarning)
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Check for available GPUs and set memory growth for the first GPU

# List all available physical devices (including GPUs)
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Print the number of available GPUs
print("Num GPUs Available: ", len(physical_devices))

# Set memory growth for the first GPU (assuming at least one GPU is available)
tf.config.experimental.set_memory_growth(physical_devices[0], True)


Num GPUs Available:  1


# **Data preparation**

In [3]:
# Change the current working directory to '/kaggle/working'
os.chdir('/kaggle/working')

# Define the source directory where the dataset is located
source_directory = '/kaggle/input/catdog-dataset/dogs-and-cats'

# Check if a directory named 'dogcat_dataset' does not exist
if os.path.isdir('dogcat_dataset') is False:
    # If 'dogcat_dataset' does not exist, copy the contents of 'source_directory' to 'dogcat_dataset'
    shutil.copytree(source_directory, 'dogcat_dataset')


In [4]:
# Define paths for training, validation, and test sets
train_path = '/kaggle/working/dogcat_dataset/train'
valid_path = '/kaggle/working/dogcat_dataset/valid'
test_path = '/kaggle/working/dogcat_dataset/test'

# Check if the directories exist and print the results
print(os.path.isdir(train_path))  # Check if 'train' directory exists
print(os.path.isdir(valid_path))  # Check if 'valid' directory exists
print(os.path.isdir(test_path))   # Check if 'test' directory exists


True
True
True


In [5]:
# Define ImageDataGenerator and generate batches of images

# Training set
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

# Validation set
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)

# Test set
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)


Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
